In [ ]:
import pandas as pd
from enceladus.workflows import TestingPipeline

worker = TestingPipeline(
    data_dir='/home/cam/Documents/database_tools/data/mimic3-data-20230220/data/',
    model_dir='heartfelt/enceladus-scripts/model-stoic-music-1:v14',
    scaler_dir='/home/cam/Documents/database_tools/data/mimic3-data-20230220/data/min_max_scaler_1677039520347015684.pkl',
)

# returns rescaled data
error, ppg, vpg, apg, abp, pred = worker.run()

In [2]:
import plotly.graph_objects as go
import pickle as pkl
import keras
import wandb
import numpy as np

def load_model(path):
    run = wandb.init()
    artifact = run.use_artifact(path, type='model')
    artifact_dir = artifact.download()
    wandb.finish()
    model = keras.models.load_model(artifact_dir)
    return model

model = load_model('heartfelt/enceladus-scripts/model-stoic-music-1:v14')

wandb: Downloading large artifact model-stoic-music-1:v14, 58.89MB. 4 files... 
wandb:   4 of 4 files downloaded.  
Done. 0:0:0.0


In [10]:
with open('/home/cam/Documents/database_tools/notebooks/test-data.pkl', 'rb') as f:
    data = pkl.load(f)

ppg, vpg, apg = data
pred = model.predict([ppg, vpg, apg])

fig = go.Figure()
fig.add_scatter(y=pred.reshape(-1))

1/1 [==============================] - 0s 13ms/step


In [8]:
pred_rescaled

array([[[-249.64847 ],
        [-243.08882 ],
        [-231.30417 ],
        [-254.16864 ],
        [-227.28383 ],
        [-234.65428 ],
        [-237.6315  ],
        [-233.91794 ],
        [-229.64703 ],
        [-228.685   ],
        [-225.28918 ],
        [-228.36253 ],
        [-224.42674 ],
        [-222.7351  ],
        [-213.87817 ],
        [-205.18925 ],
        [-197.94534 ],
        [-188.81996 ],
        [-182.18079 ],
        [-176.9079  ],
        [-173.89056 ],
        [-174.46126 ],
        [-176.37912 ],
        [-177.64961 ],
        [-173.63863 ],
        [-165.26868 ],
        [-159.89523 ],
        [-156.50146 ],
        [-153.26816 ],
        [-151.87218 ],
        [-154.2813  ],
        [-158.57721 ],
        [-165.68935 ],
        [-171.1459  ],
        [-168.80045 ],
        [-159.03671 ],
        [-148.24208 ],
        [-138.46957 ],
        [-118.47885 ],
        [ -92.35416 ],
        [ -75.186035],
        [ -59.16617 ],
        [ -40.99118 ],
        [ -

In [ ]:
import pickle as pkl

with open('/home/cam/Documents/database_tools/data/mimic3-data-20230220/data/stoic_music_1_v8_test_results.pkl', 'wb') as f:
    pkl.dump(dict(
        error=error,
        ppg=ppg,
        vpg=vpg,
        apg=apg,
        abp=abp,
        pred=pred,
    ), f)

In [ ]:
import pickle as pkl

with open('/home/cam/Documents/database_tools/data/mimic3-data-20230220/data/test_results.pkl', 'rb') as f:
    x = pkl.load(f)

# breezy-wind-80:v8

### MIMIC-III 30k Test Results (max, min method)

|   Error  | Systolic | Diastolic |
|   -:     | -:       | -:        |
| < 15mmHg | 72.6%    | 89.5%     |
| < 10mmHg | 59.1%    | 80.2%     |
| <  5mmHg | 36.1%    | 59.4%     |

### VitalDB 30k Test Results

|   Error  | Systolic | Diastolic |
|   -:     | -:       | -:        |
| < 15mmHg | %    | %     |
| < 10mmHg | %    | %     |
| <  5mmHg | %    | %     |

# breezy-wind-80:v8

### MIMIC-III 30k Test Results (peak, valley method)

|   Error  | Systolic | Diastolic |
|   -:     | -:       | -:        |
| < 15mmHg | 76.9%    | 77.9%     |
| < 10mmHg | 63.5%    | 70.4%     |
| <  5mmHg | 40.5%    | 53.9%     |

In [ ]:
import pandas as pd
import plotly.graph_objects as go

pd.options.plotting.backend = 'plotly'

fig = go.Figure()
fig.add_trace(go.Histogram(x=error['sbp_err'], name='SBP Error'))
fig.add_trace(go.Histogram(x=error['dbp_err'], name='DBP Error'))

fig.update_layout(
        barmode='overlay',
        title={
            'text': 'Model Prediction Error Histogram',
            'font': {'size': 20},
        },
        xaxis={'title': 'Error (mmHg)', 'titlefont':{'size': 15}},
        yaxis={'title': 'Number of Samples', 'titlefont':{'size': 15}},
        font={
            'family': 'Courier New, monospace',
            'color' : '#FFFFFF',
            'size'  : 12,
        },
        template='plotly_dark',
        width=800,
)
fig
fig.update_traces(opacity=0.75)

In [ ]:
import plotly.graph_objects as go
from random import randint
from scipy.signal import find_peaks

fig = go.Figure()
i = randint(0, 29999)

peaks = find_peaks(abp[i], distance=50)[0]

fig.add_scatter(x=peaks, y=abp[i][peaks], mode='markers')
fig.add_scatter(y=abp[i], name='true')
fig.add_scatter(y=pred[i], name='pred')

In [ ]:
from plotly.subplots import make_subplots

i = 18
ppg, vpg, apg, abp, pred = None

fig = make_subplots(rows=2, cols=1, start_cell='top-left')

# Add inputs
fig.add_scatter(
    y=ppg[i, :],
    name='PPG',
    row=1,
    col=1,
)
fig.add_scatter(
    y=vpg[i, :] * 10,
    name='VPG',
    row=1,
    col=1,
)
fig.add_scatter(
    y=apg[i, :] * 50,
    name='APG',
    row=1,
    col=1,
)

# Add ABP and prediction
fig.add_scatter(
    y=abp[i, :],
    name='ABP (Ground Truth)',
    row=2,
    col=1
)
fig.add_scatter(
    y=pred[i, :],
    name='Prediction',
    row=2,
    col=1,
)

fig.update_layout(
        title={
            'text': 'Model Inputs (PPG, VPG, APG) and Module Output w/ True ABP',
            'font': {'size': 20},
        },
        xaxis2={'title': 'Time (samples)', 'titlefont':{'size': 15}},
        yaxis={'title': 'Amplitude (V)', 'titlefont':{'size': 15}},
        yaxis2={'title': 'Amplitude (mmHg)', 'titlefont':{'size': 15}},
        font={
            'family': 'Courier New, monospace',
            'color' : '#FFFFFF',
            'size'  : 12,
        },
        template='plotly_dark',
        width=800,
)
fig